#Decision Tree from Scratch

In [1]:
# loading the dataset from previous NB template
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt 
import seaborn as sns
import math
iris = sns.load_dataset('iris')

iris.head()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [2]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [ ]:
# To keep it simple we convert the dataset of three classes into two classes. 

k=iris.groupby('species')
trn=pd.DataFrame()
tst=pd.DataFrame()
p=0
for g,i in k:
  if g!='setosa':
    trn=trn.append(i[:-10],ignore_index=True)
    tst=tst.append(i[-10:],ignore_index=True)

In [ ]:
trn.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')

As the features are continuous we need to convert it to categorical.

***Steps followed to build a decision tree algorithm*** 
1. If the data is continuous we need to convert it into categorical.
2. Apply the gini index or change in entropy. Here we used gini index as there were 2 classes.
3. Create a decision tree which is our aim in this algorithm.
4. Once we finished creating tree we can test it and calculate the accuracy.
   

In [ ]:
# Create a node and these are the information stored in node

class Node():
  def __init__(self):
    self.Parent=None
    self.left=None
    self.right= None
    self.cls=None
    self.attr=None

# Create a linked list just to link the root node

class List():
  def __init__(self):
    self.head=None


In [ ]:
#Creating ctual solution tree


class Tree():
  def __init__(self,trainx,trainy,testx,testy,continuous=False):
    self.trainx=trainx
    self.trainy=trainy
    self.testx=testx
    self.testy=testy
    self.nodes=[]
    self.attributes=list(self.trainx.columns)[:len(self.trainx.columns)-1]
    #self.node=[Node() for i in range(2*len(self.attributes))]
    self.node=[Node()]
    self.lst=List()
    self.lst.head=self.node[0]
    if continuous:
      self.dftr=self.preparing()
      self.dfts=self.preptst()
      #self.df.append(self.train_prep)


# The functions preptst/preparing, cat and split point converts the continuous data into categorical data.
# Here preptest is for test dataset and preparing is for training dataset. 



  def preptst(self):
    cl=self.testx.columns
    g1=self.cat(self.testx[[cl[0],'species']],cl[0])
    g2=self.cat(self.testx[[cl[1],'species']],cl[1])
    g3=self.cat(self.testx[[cl[2],'species']],cl[2])
    g4=self.cat(self.testx[[cl[3],'species']],cl[3])
    df=pd.concat([g1,g2,g3,g4,self.testx[cl[4]]],axis=1)
    return df


  def preparing(self):
    cl=self.trainx.columns
    g1=self.cat(self.trainx[[cl[0],'species']],cl[0])
    g2=self.cat(self.trainx[[cl[1],'species']],cl[1])
    g3=self.cat(self.trainx[[cl[2],'species']],cl[2])
    g4=self.cat(self.trainx[[cl[3],'species']],cl[3])
    df=pd.concat([g1,g2,g3,g4,self.trainx[cl[4]]],axis=1)
    return df
  
  def cat(self,x,a):
    N=len(x)
    x2=x.sort_values(by=a)
    x2=x2.reset_index(drop=True)
    tm=x2[a]
    d=0.1
    a1=tm[0]+d/2
    n=int((tm[N-1]-tm[0])/d)
    st=[]
    for i in range(n):
      st.append(a1+d*i)
    g=0.5
    for s in st:
      _=self.split_point(s,x,a)
      if _<g:
        g=_
        p=s
    #print(a + 'a1={},span={},p={}'.format(a1,tm[79]-tm[0],p))
    x[a]=x[a]>p 
    return x[a]

  
  # split point function takes continuous data/ attribute as input and find the point 
  # at which it can be split to make it categorical 

  def split_point(self,s,x,a):
    N=len(x)
    tm1=x[x[a]<=s]
    tm2=x[x[a]>s]
    q=len(tm1)
    b=tm1['species'].value_counts().iloc[0]
    c=tm2['species'].value_counts().iloc[0]
    #print('q={},b={},c={}'.format(q,b,c))
    c1=N-q-c
    b1=q-b
    #print(b1)
    g1= 1-((b/q)**2) -(b1/q)**2
    g2= 1-((c/(N-q))**2)-(c1/(N-q))**2
    #print('g1={},g2={}'.format(g1,g2))
    return (g1+g2)/2



  #  Following are steps for writing actual solution algorithm 



# The gini function takes the attribute and dataframe as input and returns the gini value
# if we split the given dataframe by the attribute

  def gini(self,attri,df):
    gini=[]
    class1,class2=df['species'].unique()
    for i in attri:
      #print('\n {} \n'.format(i))
      if len(df[i].value_counts())>1:
        df1=df[df[i]==True]
        df2=df[df[i]==False]
        tmp=df['species'].value_counts()
        l=len(df)
        gini_parent=1-(tmp[class1]/l)**2-(tmp[class2]/l)**2
        tmp1=df1['species'].value_counts()
        #print('tmp1',tmp1)
        tmp2=df2['species'].value_counts()
        #print('tmp2',tmp2)
        l1=len(df1)
        l2=len(df2)
        if len(tmp1)==1:
          gini_child1=0
        else:
          gini_child1=(1-(tmp1[class1]/l1)**2-(tmp1[class2]/l1)**2)
        if len(tmp2)==1:
          gini_child2=0
        else:
          gini_child2=(1-(tmp2[class1]/l2)**2-(tmp2[class2]/l2)**2)

        gini_child=(gini_child1+gini_child2)/2
        gain=gini_parent-gini_child
        gini.append(gain)
      else:
        gini.append(0) 
    return gini


# This function build the tree. Creates new node dynamically as tree grows. 
# Besides all other function this particular function is made somewhat generalized.
# i.e for any no of attributes it will create a solution tree  

  def create_sol(self):
    cnt=0
    self.nodes.append(self.node[0])
    df=[]
    df.append(self.dftr)
    while len(df)!=0:
      for i in range(2):
        self.node.append(Node())
      #print(cnt,'\n')
      d=df.pop(0)
      #print(d['species'])
      #print(d['species'].max())
      n=self.nodes.pop(0)
      k=len(d['species'].unique())
      if k==2:
        gini=self.gini(self.attributes,d)
      else:
        gini=[0,0,0,0]
      #print('gini_gain = {}\n\n most frequent species:\n {}'.format(gini,d['species'].value_counts().index[0]))
      if max(gini)>0:
        #print('in block')
        attri=self.attributes[gini.index(max(gini))]
        n.attr=attri
        df1=d[d[attri]==True]
        df2=d[d[attri]==False]
        df.append(df1)
        df.append(df2)
        n.left=self.node[cnt+1]
        self.nodes.append(self.node[cnt+1])
        n.right=self.node[cnt+2]
        self.nodes.append(self.node[cnt+2])
      #  self.node[cnt+1].parent=n
       # self.node[cnt+2].parent=n
        cnt=cnt+2
      else:
        n.cls=d['species'].value_counts().index[0]


# Following function is to calculate the accuracy. Based on the tree builded 


  def acc(self):
    n=len(self.dftr)
    t=self.dftr
    pr=[]
    for i in range(n):
      ex=t.iloc[i]
      tmp=self.lst.head
      cl=tmp.cls
      #print('cl out=',cl)
      while cl==None:
        #print('cl=',cl)
        if ex[tmp.attr]==True:
          #print('a')
          tmp=tmp.left
          cl=tmp.cls
        elif ex[tmp.attr]==False:
          #print('b')
          tmp=tmp.right
          cl=tmp.cls
        else:
          print('something went wrong')
      pr.append(cl)
    
    pr=pd.Series(pr)
    #print(self.trainy)
    #print('pr=\n',pr)
    x=pr==self.trainy
    #print('x=',x)

    trn_acc=x.value_counts()[True]/len(pr)
    

    n=len(self.dfts)
    t=self.dfts
    pr=[]
    for i in range(n):
      ex=t.iloc[i]
      tmp=self.lst.head
      cl=tmp.cls
      while cl==None:
        if ex[tmp.attr]==True:
          tmp=tmp.left
          cl=tmp.cls
        elif ex[tmp.attr]==False:
          tmp=tmp.right
          cl=tmp.cls
        else:
          print('something went wrong')
      pr.append(cl)
    
    pr=pd.Series(pr)
    x=pr==self.testy
    tst_acc=x.value_counts()[True]/len(pr)

    print('Training accuracy = {} \n Test Accuracy = {}'.format(trn_acc*100,tst_acc*100))


  

In [ ]:
# Initialize the class
trny=trn['species']
tsty=tst['species']
tr=Tree(trn,trny,tst,tsty,True)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
# Create the solution tree
tr.create_sol()

In [ ]:
# Calculate the accuracy
tr.acc()

Training accuracy = 93.75 
 Test Accuracy = 85.0


#Using Sklearn 

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
cross_val_score(clf, trn[at[0:4]], trny, cv=10)

array([1.   , 0.875, 1.   , 1.   , 0.875, 0.875, 0.875, 0.875, 0.875,
       0.875])